<a href="https://colab.research.google.com/github/saivaruntejamudumba/Python-Scripts/blob/master/Tabular_Data_Classification_FastAi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!wget https://raw.githubusercontent.com/wshuyi/demo-customer-churn-ann/master/customer_churn.csv

--2019-04-23 06:56:08--  https://raw.githubusercontent.com/wshuyi/demo-customer-churn-ann/master/customer_churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 684858 (669K) [text/plain]
Saving to: ‘customer_churn.csv’

customer_churn.csv  100%[===================>] 668.81K  --.-KB/s    in 0.04s   

2019-04-23 06:56:08 (18.0 MB/s) - ‘customer_churn.csv’ saved [684858/684858]



In [0]:
import pandas as pd

In [0]:
df=pd.read_csv('customer_churn.csv')

In [17]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [18]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [0]:
import fastai
from fastai.tabular import * 

In [0]:
procs = [FillMissing, Categorify, Normalize]
valid_idx = range(len(df)-2000, len(df))

In [0]:
dep_var = 'Exited'
cat_names = ['Geography','Gender','HasCrCard','IsActiveMember']
cont_names = ['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']

In [26]:
data = TabularDataBunch.from_df('.', df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=cat_names, cont_names=cont_names)
print(data.train_ds.cont_names)

['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']


In [27]:
(cat_x,cont_x),y = next(iter(data.train_dl))
for o in (cat_x, cont_x, y): print(to_np(o[:5]))

[[1 2 2 1]
 [3 2 1 2]
 [1 2 2 1]
 [1 2 2 1]
 [3 1 2 1]]
[[-1.044766e+00 -9.454536e-01 -6.978111e-01 -1.230944e+00  8.094631e-01  7.977851e-01]
 [-1.854781e-03  1.961831e-01  1.382361e+00 -1.230944e+00  8.094631e-01  1.588921e+00]
 [ 1.010079e+00  2.003775e+00 -1.737897e+00  1.210678e+00 -9.055011e-01  4.614550e-01]
 [ 2.356398e-01  5.910349e-03  6.889704e-01  8.743979e-02  8.094631e-01  1.624882e-01]
 [-2.376801e+00  1.528093e+00 -3.511157e-01  2.419137e-01 -9.055011e-01  7.765748e-01]]
[0 0 1 0 1]


In [28]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)
learn.fit_one_cycle(5, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.428609,0.367776,0.848500,00:01
1,0.363627,0.353001,0.857000,00:01
2,0.354703,0.354092,0.858500,00:01
3,0.343215,0.344407,0.853500,00:01
4,0.341340,0.348572,0.856500,00:01


In [29]:
learn.predict(df.iloc[0])

(Category 0, tensor(0), tensor([0.6144, 0.3856]))